In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
tmp = pd.read_csv(r"C:\Users\ZQH\Desktop\上法兰西\fr.csv")#导入数据集

df =pd.DataFrame(tmp.y.values,columns=["y"])#载入负荷数据
df.index = pd.to_datetime(tmp.ds.values)#制作index，时间序列格式
df = df[df.index<pd.to_datetime("2020-1-1")]#只要2020年以前的

df = df.resample("D").max()

In [5]:
def func_make_windows(df,memoryW=4,forecastW=0):
    tmp = df.copy()#制作窗口数据
    tmp["114.514"]=114.514
    tmp.columns=["tp0","tn0"]
    tmp["tn0"]=tmp["tp0"]
    
    for i in range(1,memoryW+1):
        tmp["tn"+str(i)]=np.append(None,tmp["tn"+str(i-1)].values)[:len(tmp)]
    
    for i in range(1,forecastW+1):
        tmp["tp"+str(i)]=np.append(tmp["tp"+str(i-1)].values,None)[1:]
        
    tmp2 = []
    for i in range(memoryW+1):
        tmp2.append("tn"+str(memoryW-i))

    X_label=tmp2.copy()
    
    y_label=[]
    for i in range(forecastW+1):
        tmp2.append("tp"+str(i))
    
    df = tmp.loc[:,tmp2].dropna().astype("float")
    
    train = df[df.index>=pd.to_datetime("2016-1-1")]
    train = train[train.index<pd.to_datetime("2018-1-1")]
    
    test  = df[df.index>=pd.to_datetime("2019-1-1")]
    test = test[test.index<pd.to_datetime("2020-1-1")]
    
    X_train = train.loc[:,X_label].drop("tn0",axis=1)
    y_train = train.drop(X_label,axis=1)#train["tn0"]
    
    X_test = test.loc[:,X_label].drop("tn0",axis=1)
    y_test = test.drop(X_label,axis=1)#test["tn0"]
    
    meta_df = (X_train,y_train,X_test,y_test)
    
    return meta_df

In [6]:
def func_make_windows2(df,memoryW=4,forecastW=0):
    tmp = df.copy()#制作窗口数据
    tmp["114.514"]=114.514
    tmp.columns=["tp0","tn0"]
    tmp["tn0"]=tmp["tp0"]
    
    for i in range(1,memoryW+1):
        tmp["tn"+str(i)]=np.append(None,tmp["tn"+str(i-1)].values)[:len(tmp)]
    
    for i in range(1,forecastW+1):
        tmp["tp"+str(i)]=np.append(tmp["tp"+str(i-1)].values,None)[1:]
        
    tmp2 = []
    for i in range(memoryW+1):
        tmp2.append("tn"+str(memoryW-i))

    X_label=tmp2.copy()
    
    y_label=[]
    for i in range(forecastW+1):
        tmp2.append("tp"+str(i))
    
    df = tmp.loc[:,tmp2].dropna().astype("float")
    
    train = df[df.index>=pd.to_datetime("2016-1-1")]
    train = train[train.index<pd.to_datetime("2018-1-1")]
    
    test  = df[df.index>=pd.to_datetime("2018-1-1")]
    test = test[test.index<pd.to_datetime("2019-1-1")]
    
    X_train = train.loc[:,X_label].drop("tn0",axis=1)
    y_train = train.drop(X_label,axis=1)#train["tn0"]
    
    X_test = test.loc[:,X_label].drop("tn0",axis=1)
    y_test = test.drop(X_label,axis=1)#test["tn0"]
    
    meta_df = (X_train,y_train,X_test,y_test)
    #y_test.iloc[:,0].plot()
    
    
    return meta_df

In [7]:
def time_add(X_train):
    tmp=X_train.copy()
    tmp["ds"] = tmp.index
    #tmp["minuteofhour"] = tmp["ds"].dt.minute
    #tmp["hourofday"] = tmp["ds"].dt.hour
    tmp["dayofweek"] = tmp["ds"].dt.dayofweek
    tmp["dayofmonth"] = tmp["ds"].dt.day
    tmp["dayofyear"] = tmp["ds"].dt.dayofyear
    tmp["weekofyear"] = tmp["ds"].dt.weekofyear
    tmp["monthofyear"] = tmp["ds"].dt.month
    tmp = tmp.drop("ds",axis=1)
    return tmp

# 对比模型

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as mae

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRFRegressor,XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

mds={}
mds["lrs"]   = LinearRegression()
mds["tree"]   = DecisionTreeRegressor(random_state=615)
mds["extra"]  = ExtraTreeRegressor(random_state=615)
mds["rbfsvr"] = SVR(C=1000,)
mds["knn"]    = KNeighborsRegressor()
mds["mlp3"]   = MLPRegressor(hidden_layer_sizes=(256,128,64),random_state=615)#(,32)
mds["ada100"] = AdaBoostRegressor(random_state=615,n_estimators=100)
mds["pgbdt"]  =  GradientBoostingRegressor(random_state=615,learning_rate = 0.30)
mds["xgbrf"]   = XGBRFRegressor(random_state=615)
mds["xgbt"]   = XGBRegressor(random_state=615,)





In [9]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2
from sklearn.metrics import max_error as maxe
from sklearn.metrics import median_absolute_error as med
from sklearn.metrics import mean_absolute_percentage_error as mape

results_mae={}
results_r2={}
results_max={}
results_med={}
results_mape={}
for i in (3,7,14,28):
    X_train,y_train,X_test,y_test = func_make_windows(df,memoryW=i)
    for j in mds.keys():
        
        print(j,end="")
        mds[j].fit(time_add(X_train),y_train)
        results_mae[f"{j}+{i}"]=mae(y_test,mds[j].predict(time_add(X_test)))
        results_r2[f"{j}+{i}"]=r2(y_test,mds[j].predict(time_add(X_test)))
        results_max[f"{j}+{i}"]=maxe(y_test,mds[j].predict(time_add(X_test)))
        results_med[f"{j}+{i}"]=med(y_test,mds[j].predict(time_add(X_test)))
        results_mape[f"{j}+{i}"]=mape(y_test,mds[j].predict(time_add(X_test)))

lrstreeextrarbfsvrknnmlp3ada100pgbdtxgbrfxgbtlrstreeextrarbfsvrknnmlp3ada100pgbdtxgbrfxgbtlrstreeextrarbfsvrknnmlp3ada100pgbdtxgbrfxgbtlrstreeextrarbfsvrknnmlp3ada100pgbdtxgbrfxgbt

In [10]:
_result = pd.DataFrame(data = results_mae.values(),index=results_mae.keys(),columns=["mae"])
_result["r2"]=pd.DataFrame(data = results_r2.values(),index=results_r2.keys(),columns=["r2"])
_result["max"]=pd.DataFrame(data = results_max.values(),index=results_max.keys(),columns=["max"])
_result["med"]=pd.DataFrame(data = results_med.values(),index=results_med.keys(),columns=["med"])
_result["mape"]=pd.DataFrame(data = results_mape.values(),index=results_mape.keys(),columns=["mape"])

In [11]:
_result

,mae,r2,max,med,mape
lrs+3,202.800100,0.930713,974.693659,152.149391,0.031936
tree+3,255.827397,0.883545,1424.000000,188.000000,0.039983
extra+3,295.654795,0.831575,2013.000000,193.000000,0.046567
rbfsvr+3,303.880290,0.842979,1448.320787,224.267918,0.048048
knn+3,269.076164,0.875062,1492.400000,199.600000,0.042091
mlp3+3,268.280318,0.888473,1505.517280,212.287698,0.041856
ada100+3,249.246847,0.908236,1254.600000,213.450549,0.040756
pgbdt+3,189.550697,0.936102,1189.502668,137.524121,0.029655
xgbrf+3,189.700995,0.935467,1293.531250,138.049805,0.029931
xgbt+3,193.570275,0.932773,1174.250000,138.835938,0.030360


# better model

In [12]:
import optuna
import sklearn.metrics
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score


def objective(trial):

    param = {
        # regularization
        "alpha": trial.suggest_float("alpha", 0.01, 1.00, log=False),
        "eta": trial.suggest_float("eta", 0.01,0.20, log=False),
        
        
        # 
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "min_child_weight": trial.suggest_float("min_child_weight", 0.50, 0.60, log=False),
        "gamma": trial.suggest_float("gamma", 1e-10, 1, log=False),
     

        #sampling 
        "subsample": trial.suggest_float("subsample", 0.90, 1.00, log=False),
        "colsample_bytree": trial.suggest_float("colsample_bytree",   0.90, 1.00,   log=False),#0.9, 0.95
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.90, 1.00,   log=False),
        "colsample_bynode": trial.suggest_float("colsample_bynode",   0.90, 1.00,   log=False),
        
 
    }
    
    
    md = XGBRegressor(random_state=615,**param)
    md.fit(X_train_cell,y_train)
    _r3 = mape(md.predict(X_vaild_cell),y_vaild)

    
    return _r3

## better3

In [13]:
_,_,X_vaild,y_vaild = func_make_windows2(df=df,memoryW=3)
X_train,y_train,X_test,y_test = func_make_windows(df=df,memoryW=3)

X_train_cell = time_add(X_train)
X_test_cell = time_add(X_test)
X_vaild_cell= time_add(X_vaild)

study3 = optuna.create_study(direction="minimize")
study3.optimize(objective, n_trials=500, timeout=300)

xgb_md = XGBRegressor(random_state=615,**study3.best_params)
xgb_md.fit(time_add(X_train),y_train)
hat3 = pd.DataFrame(xgb_md.predict(time_add(X_test)),index=y_test.index)
better3={}
better3["mae"]=mae(y_test,hat3)
better3["r2"]=r2(y_test,hat3)
better3["max"]=maxe(y_test,hat3)
better3["med"]=med(y_test,hat3)
better3["mape"]=mape(y_test,hat3)
better3=pd.DataFrame(better3.values(),columns=["better3"],index=better3.keys())
better3

[I 2022-04-04 23:37:36,598] A new study created in memory with name: no-name-84b611bf-cb4c-42c6-bc23-4bb6bd84c0c0
[I 2022-04-04 23:37:36,675] Trial 0 finished with value: 0.028678485857859752 and parameters: {'alpha': 0.8696948957260008, 'eta': 0.05250066917035614, 'max_depth': 5, 'min_child_weight': 0.542873626966067, 'gamma': 0.5158425767175386, 'subsample': 0.9595310591164431, 'colsample_bytree': 0.9607923709825835, 'colsample_bylevel': 0.9763472827515801, 'colsample_bynode': 0.9851159764142479}. Best is trial 0 with value: 0.028678485857859752.
[I 2022-04-04 23:37:36,745] Trial 1 finished with value: 0.029488669155683687 and parameters: {'alpha': 0.3057928611728769, 'eta': 0.17292434545873406, 'max_depth': 4, 'min_child_weight': 0.5874460248130158, 'gamma': 0.1378173662089658, 'subsample': 0.9874727609754731, 'colsample_bytree': 0.9825265153022498, 'colsample_bylevel': 0.9965350039980619, 'colsample_bynode': 0.9850352189915501}. Best is trial 0 with value: 0.028678485857859752.
[I 

,better3
mae,166.020068
r2,0.947104
max,1183.269531
med,101.641113
mape,0.026060


## 7

In [14]:
_,_,X_vaild,y_vaild = func_make_windows2(df=df,memoryW=7)
X_train,y_train,X_test,y_test = func_make_windows(df=df,memoryW=7)

X_train_cell = time_add(X_train)
X_test_cell = time_add(X_test)
X_vaild_cell= time_add(X_vaild)

study7 = optuna.create_study(direction="minimize")
study7.optimize(objective, n_trials=500, timeout=300)

xgb_md7 = XGBRegressor(random_state=615,**study7.best_params)
xgb_md7.fit(time_add(X_train),y_train)
hat7 = pd.DataFrame(xgb_md7.predict(time_add(X_test)),index=y_test.index)

better7={}
better7["mae"]=mae(y_test,hat7)
better7["r2"]=r2(y_test,hat7)
better7["max"]=maxe(y_test,hat7)
better7["med"]=med(y_test,hat7)
better7["mape"]=mape(y_test,hat7)
better7=pd.DataFrame(better7.values(),columns=["better7"],index=better7.keys())

[I 2022-04-04 23:38:43,847] A new study created in memory with name: no-name-a8d621d5-1006-45ed-8433-68e896abbe79
[I 2022-04-04 23:38:43,900] Trial 0 finished with value: 0.031478739976253056 and parameters: {'alpha': 0.17839019849558588, 'eta': 0.15933209416208566, 'max_depth': 2, 'min_child_weight': 0.5765628186592714, 'gamma': 0.6535652169649327, 'subsample': 0.9691333263403947, 'colsample_bytree': 0.9603000154784089, 'colsample_bylevel': 0.9558277984376625, 'colsample_bynode': 0.9507064575004812}. Best is trial 0 with value: 0.031478739976253056.
[I 2022-04-04 23:38:43,965] Trial 1 finished with value: 0.029709728730540275 and parameters: {'alpha': 0.7332243344694103, 'eta': 0.12920586111364826, 'max_depth': 3, 'min_child_weight': 0.5449274339145576, 'gamma': 0.8622979754200417, 'subsample': 0.9975457444664328, 'colsample_bytree': 0.9199716307788051, 'colsample_bylevel': 0.9373110220003438, 'colsample_bynode': 0.9250449956737804}. Best is trial 1 with value: 0.029709728730540275.
[

## 14

In [15]:
_,_,X_vaild,y_vaild = func_make_windows2(df=df,memoryW=14)
X_train,y_train,X_test,y_test = func_make_windows(df=df,memoryW=14)

X_train_cell = time_add(X_train)
X_test_cell = time_add(X_test)
X_vaild_cell= time_add(X_vaild)

study14 = optuna.create_study(direction="minimize")
study14.optimize(objective, n_trials=500, timeout=300)

xgb_md14 = XGBRegressor(random_state=615,**study14.best_params)
xgb_md14.fit(time_add(X_train),y_train)
hat14 = pd.DataFrame(xgb_md14.predict(time_add(X_test)),index=y_test.index)

better14={}
better14["mae"]=mae(y_test,hat14)
better14["r2"]=r2(y_test,hat14)
better14["max"]=maxe(y_test,hat14)
better14["med"]=med(y_test,hat14)
better14["mape"]=mape(y_test,hat14)
better14=pd.DataFrame(better14.values(),columns=["better14"],index=better14.keys())

[I 2022-04-04 23:39:50,071] A new study created in memory with name: no-name-54b67e1d-0d31-4503-bf9a-a4a02de3d360
[I 2022-04-04 23:39:50,133] Trial 0 finished with value: 0.039391218252308444 and parameters: {'alpha': 0.828821540637789, 'eta': 0.039386088540456866, 'max_depth': 2, 'min_child_weight': 0.5164831226983262, 'gamma': 0.20203823832673995, 'subsample': 0.9714022029816102, 'colsample_bytree': 0.9177121663895409, 'colsample_bylevel': 0.9398940442523925, 'colsample_bynode': 0.9643537230259818}. Best is trial 0 with value: 0.039391218252308444.
[I 2022-04-04 23:39:50,201] Trial 1 finished with value: 0.030402926025008454 and parameters: {'alpha': 0.8052589543476043, 'eta': 0.09422485311904362, 'max_depth': 3, 'min_child_weight': 0.5317495692724215, 'gamma': 0.5828773468503285, 'subsample': 0.970890704716164, 'colsample_bytree': 0.9102225916617015, 'colsample_bylevel': 0.9242885684978202, 'colsample_bynode': 0.9482175317181433}. Best is trial 1 with value: 0.030402926025008454.
[I

## 28

In [16]:
_,_,X_vaild,y_vaild = func_make_windows2(df=df,memoryW=28)
X_train,y_train,X_test,y_test = func_make_windows(df=df,memoryW=28)

X_train_cell = time_add(X_train)
X_test_cell = time_add(X_test)
X_vaild_cell= time_add(X_vaild)

study28 = optuna.create_study(direction="minimize")
study28.optimize(objective, n_trials=500, timeout=300)

xgb_md28 = XGBRegressor(random_state=615,**study28.best_params)
xgb_md28.fit(time_add(X_train),y_train)
hat28 = pd.DataFrame(xgb_md28.predict(time_add(X_test)),index=y_test.index)

better28={}
better28["mae"]=mae(y_test,hat28)
better28["r2"]=r2(y_test,hat28)
better28["max"]=maxe(y_test,hat28)
better28["med"]=med(y_test,hat28)
better28["mape"]=mape(y_test,hat28)
better28=pd.DataFrame(better28.values(),columns=["better28"],index=better28.keys())

[I 2022-04-04 23:40:48,642] A new study created in memory with name: no-name-b7eb8ca2-bb5f-4d79-97e8-7a462804ed75
[I 2022-04-04 23:40:48,726] Trial 0 finished with value: 0.03218416376632968 and parameters: {'alpha': 0.08788592569508678, 'eta': 0.17842640455003558, 'max_depth': 2, 'min_child_weight': 0.5933407288289148, 'gamma': 0.9273201891062828, 'subsample': 0.917753138915606, 'colsample_bytree': 0.9314926194785452, 'colsample_bylevel': 0.9759330199864056, 'colsample_bynode': 0.9546011181321448}. Best is trial 0 with value: 0.03218416376632968.
[I 2022-04-04 23:40:48,803] Trial 1 finished with value: 0.03199243398630607 and parameters: {'alpha': 0.5159861663913924, 'eta': 0.10302777240414615, 'max_depth': 2, 'min_child_weight': 0.5664103604145896, 'gamma': 0.9371548074283165, 'subsample': 0.9319928034572074, 'colsample_bytree': 0.9941584077500654, 'colsample_bylevel': 0.9478358255717878, 'colsample_bynode': 0.9854218156551777}. Best is trial 1 with value: 0.03199243398630607.
[I 202

In [17]:
res_tpe = better3.copy()
res_tpe.columns=["better+3"]
res_tpe["better+7"]=better7.copy()
res_tpe["better+14"]=better14.copy()
res_tpe["better+28"]=better28.copy()
res_tpe = res_tpe.T
_result = _result.append(res_tpe)
_result

,mae,r2,max,med,mape
lrs+3,202.800100,0.930713,974.693659,152.149391,0.031936
tree+3,255.827397,0.883545,1424.000000,188.000000,0.039983
extra+3,295.654795,0.831575,2013.000000,193.000000,0.046567
rbfsvr+3,303.880290,0.842979,1448.320787,224.267918,0.048048
knn+3,269.076164,0.875062,1492.400000,199.600000,0.042091
mlp3+3,268.280318,0.888473,1505.517280,212.287698,0.041856
ada100+3,249.246847,0.908236,1254.600000,213.450549,0.040756
pgbdt+3,189.550697,0.936102,1189.502668,137.524121,0.029655
xgbrf+3,189.700995,0.935467,1293.531250,138.049805,0.029931
xgbt+3,193.570275,0.932773,1174.250000,138.835938,0.030360


In [19]:
#_result.to_excel(r"C:\Users\ZQH\Desktop\上法兰西\data\tpe_res.xlsx")

In [20]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [25]:
X_train_p3,y_train,X_test_p3,y_test = func_make_windows(df,memoryW=3)
X_train_p3 = time_add(X_train_p3)
X_test_p3  = time_add(X_test_p3)

xgb_p3 = XGBRegressor(random_state=615)
xgb_p3.fit(X_train_p3,y_train)

hatp3 = pd.DataFrame(data=xgb_p3.predict(X_test_p3),index=y_test.index)


In [26]:
table3data = y_test.copy()
table3data["tpe3"]=hat3
table3data["xgb3"]=hatp3
table3data.columns=["real values","tpe-xgboost","xgboost"]
#table3data.to_excel(r"C:\Users\ZQH\Desktop\上法兰西\data\3dif.xlsx")

In [27]:
fis={}
c={}
for i in [3,7,14,28]:
    fis[i] = XGBRegressor(random_state=615)
    X_train_fs,y_train_fs,_,_ = func_make_windows(df,memoryW=i)
    
    fis[i].fit(time_add(X_train_fs),y_train_fs)
    c[i]=pd.DataFrame(data=fis[i].feature_importances_,index=time_add(X_train_fs).columns,columns=[str(i)])
    

In [36]:
s3 = study3.trials_dataframe()
s3.to_excel(r"C:\Users\ZQH\Desktop\上法兰西\data\s3.xlsx")

In [53]:
pd.DataFrame(xgb_md.feature_importances_,index=c[3].index).to_excel(r"C:\Users\ZQH\Desktop\上法兰西\data\b3.xlsx")
pd.DataFrame(xgb_md7.feature_importances_,index=c[7].index).to_excel(r"C:\Users\ZQH\Desktop\上法兰西\data\b7.xlsx")
#c[3].to_excel(r"C:\Users\ZQH\Desktop\上法兰西\data\x3.xlsx")
#c[7].to_excel(r"C:\Users\ZQH\Desktop\上法兰西\data\x7.xlsx")